# Inspecting smart contracts with PyTezos

In [ ]:
import sys
assert sys.version_info.major == 3 and sys.version_info.minor >= 6

In [ ]:
import os
sys.path.insert(1, os.path.abspath('..'))

## Searching for origination

As was shown in the previous post, thanks to Tezos statefulness we can rather quickly look for transactions of a special kind.  
First af all let's try to find a smart contract origination.

In [ ]:
from pytezos.rpc import alphanet, mainnet
from pytezos.rpc.search import SearchChain

In [ ]:
sc = SearchChain.from_chain(mainnet.main)

We will look for one of the TezVote smart contracts.  
The algorithm checks for the contract existence, before the origination RPC query have to return 404.

In [ ]:
origination = sc.find_contract_origination_operation('KT1ExvG3EjTrvDcAU7EqLNb77agPa5u6KvnY')

In [ ]:
origination()

## Finding all storage altering transactions

We can also use step&bisect algo to retrieve all transactions that have changed the SC storage.

In [ ]:
for op in sc.find_storage_change_operations('KT1ExvG3EjTrvDcAU7EqLNb77agPa5u6KvnY', origination_level=293492):
    break

In [ ]:
op()

## Decoding storage

Allright, now let's view some data, i'll take some random SC from alphanet for demonstration

In [ ]:
alphanet.context.contracts['KT19iGCL4YrVpT6ezEzbDH37Yxbas8jWQz4s'].storage()

In [ ]:
alphanet.context.contracts['KT1VnRY6UuWH89P8DQtC7Sd178jkckff8o8d'].storage()

In [ ]:
alphanet.context.contracts['KT1FEDVALSfQLZwVZbF1hRxJ9c8MTPe7azCZ'].storage()

Okay that won't work, let's do some magic:

In [ ]:
alphanet.context.contracts['KT19iGCL4YrVpT6ezEzbDH37Yxbas8jWQz4s'].decode_storage()

In [ ]:
alphanet.context.contracts['KT1VnRY6UuWH89P8DQtC7Sd178jkckff8o8d'].decode_storage()

In [ ]:
alphanet.context.contracts['KT1FEDVALSfQLZwVZbF1hRxJ9c8MTPe7azCZ'].decode_storage()

Oh yeah, much better :)  
Note that we can handle both annotated and not annotated storage.

## Encoding storage

We can easily encode storage data back, this functional is quite useful for unit testing.

In [ ]:
contract = alphanet.context.contracts['KT1VnRY6UuWH89P8DQtC7Sd178jkckff8o8d']

Let's look at the schema first

In [ ]:
contract.storage_schema()

In [ ]:
contract.encode_storage({
    'accounts': {},
    'version': 2,
    'totalSupply': 100000000,
    'decimals': 8,
    'name': 'AbcCoin',
    'symbol': 'ABC',
    'owner': 'KT1GE2AZhazRxGsAjRVkQccHcB2pvANXQWd7'
})

## Decoding transaction parameters

Great! But what about parameters we call smart contracts with?  
Take some complicated example

In [ ]:
content = alphanet.blocks[216905].operations.managers.contents()[0]

In [ ]:
contract = alphanet.context.contracts['KT1FU74GimCeEVRAEZGURb6TWU8jK1N6zFJy']

In [ ]:
content['parameters']

In [ ]:
contract.decode_parameters(content['parameters'])

As you can see, we combine type and field annotations from the code with the transaction data.

## Encoding transaction parameters

If we do not know exactly the SC interface, we can look at the internal schema representation:

In [ ]:
contract.parameter_schema()

In [ ]:
contract.encode_parameters({'Redeem': {'rdHSec': '12', 'rdSec': '34'}})

## Visualize storage diff

In [ ]:
storage_1 = mainnet.blocks[328451].context.contracts['KT1ExvG3EjTrvDcAU7EqLNb77agPa5u6KvnY'].decode_storage()
storage_1

In [ ]:
storage_2 = mainnet.blocks[334288].context.contracts['KT1ExvG3EjTrvDcAU7EqLNb77agPa5u6KvnY'].decode_storage()
storage_2

In [ ]:
from pytezos.tools.diff import generate_jsondiff_html

In [ ]:
generate_jsondiff_html(storage_1, storage_2, output_path='storage_diff.html')

[view diff](./storage_diff.html)

This is pretty what we need when analyzing a smart contract behaviour, an input and side effects.

## Accessing BigMap

In [ ]:
contract.big_map_get('52c5bcbf9cb4dcaacd8689b42726c6f11e6eb575ade913923b6b1420b5b65eb9')

We can also decode big map delta from the transaction

In [ ]:
big_map_diff = alphanet.blocks[216895].operations.contents('transaction')[0]['metadata']['operation_result']['big_map_diff']
big_map_diff

In [ ]:
contract.big_map_diff_decode(big_map_diff)

## Bonus: parsing .tz files

You can also initialize a `Contract` instance from Micheline, Michelson source string or file.

In [ ]:
source = '''
parameter nat;
storage (map address nat);
code { DUP ;
       CDR ;
       SWAP ;
       CAR ;
       DUP ;
       PUSH nat 2 ;
       { { COMPARE ; GE } ; IF {} { { UNIT ; FAILWITH } } } ;
       SOME ;
       SENDER ;
       UPDATE ;
       NIL operation ;
       PAIR }
'''

In [ ]:
from pytezos.rpc.contract import Contract

In [ ]:
contract = Contract.from_string(source)
contract